# Sentiment Analysis

## Loading Classifier

In [2]:
import torch
from transformers import pipeline
import pandas as pd
import torch
import datasets
from datasets import Dataset
from transformers.pipelines.pt_utils import KeyDataset
from tqdm.auto import tqdm

if torch.cuda.is_available():
    print('Using GPU...')
    classifier = pipeline("text-classification", model="ProsusAI/finbert", device=0, )
else:
    print('Using CPU...')
    classifier = pipeline("text-classification", model="ProsusAI/finbert", device=-1)

Using GPU...


config.json:   0%|          | 0.00/758 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/252 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [21]:
def save_labels(df, output, filename):
    positive_scores = []
    negative_scores = []
    neutral_scores = []
    final_labels = []

    for labels in output:
        positive_scores.append(labels[0]['score'])
        negative_scores.append(labels[1]['score'])
        neutral_scores.append(labels[2]['score'])
        final_labels.append(max(labels, key=lambda x: x['score'])['label'])

    df["positive"] = positive_scores
    df["negative"] = negative_scores
    df["neutral"] = neutral_scores
    df["labels"] = final_labels

    df.to_csv(filename, index=False)

In [32]:
out = classifier("STOCK IS RISING FAST", top_k=None)
print(out)
print(max(out, key=lambda x: x['score'])['label'])

[{'label': 'positive', 'score': 0.8715260028839111}, {'label': 'neutral', 'score': 0.09827440232038498}, {'label': 'negative', 'score': 0.030199550092220306}]
positive


## Analyzing News Headlines

In [3]:
## Loading csv
news_df = pd.read_csv('nvda_news_headlines.csv')

In [26]:
## Getting sentiments
news_output = classifier(news_df["title"].tolist(), top_k=None)

6266


In [29]:
## Saving to csv
save_labels(news_df, news_output, "nvda_news_sentiment.csv")

## Analyzing StockTwits

In [25]:
## Loading csv
stocktwits_df = pd.read_csv('nvda_stocktwits.csv')
stocktwits_dataset = Dataset.from_pandas(stocktwits_df)

In [ ]:
## Getting sentiments
stocktwits_output = []
for out in tqdm(classifier(KeyDataset(stocktwits_dataset, " text"), batch_size=32, top_k=None), total=len(stocktwits_dataset)):
    stocktwits_output.append(out)

In [19]:
print(len(stocktwits_output))

399584


In [22]:
save_labels(pd.DataFrame(), stocktwits_output, "nvda_stocktwits_sentiment_temp.csv")

In [23]:
stocktwits_df = stocktwits_df.tail(184597)
stocktwits_dataset = Dataset.from_pandas(stocktwits_df)
for out in tqdm(classifier(KeyDataset(stocktwits_dataset, " text"), batch_size=32, top_k=None, truncation=True), total=len(stocktwits_dataset)):
    stocktwits_output.append(out)

  0%|          | 0/184597 [00:00<?, ?it/s]

In [26]:
print(len(stocktwits_output) == len(stocktwits_df))

True


In [27]:
## Saving to csv
save_labels(stocktwits_df, stocktwits_output, "nvda_stocktwits_sentiment.csv")